In [1]:
# config.ipynb cell
import os
import sys
import supabase

# Dynamically add the parent directory to the Python path
current_dir = os.path.dirname(os.path.abspath(__name__))
parent_dir = os.path.dirname(current_dir)
sys.path.append(parent_dir)

import uuid
import random
import json
from faker import Faker
from datetime import datetime
from supabase import create_client, Client
from src.api.config import settings

# Supabase credentials
SUPABASE_URL = settings.SUPABASE_URL
SUPABASE_KEY = settings.SUPABASE_ANON_KEY
client = create_client(SUPABASE_URL, SUPABASE_KEY)


✅ Step 2: Get Distinct owner_user_ids from properties

In [2]:
response = client.table("properties").select("owner_user_id").execute()
owner_ids = list({item["owner_user_id"] for item in response.data})
print(f"{len(owner_ids)} unique owner_user_ids found.")


642 unique owner_user_ids found.


✅ Step 3: Get Existing Landlord Profiles

In [3]:
existing = client.table("landlord_profile").select("user_id").in_("user_id", owner_ids).execute()
existing_user_ids = {item["user_id"] for item in existing.data}

# Filter only new ones
new_owner_ids = [uid for uid in owner_ids if uid not in existing_user_ids]
print(f"{len(new_owner_ids)} new landlord profiles to create.")


642 new landlord profiles to create.


🧪 Step-by-Step Jupyter Implementation (Supabase Client)

✅ Step 1: Fetch all properties and group by owner_user_id

In [4]:
from collections import defaultdict
from datetime import datetime
from faker import Faker
import random

faker = Faker()

# Fetch all properties
response = client.table("properties").select("*").limit(1000).execute()
properties = response.data

# Group by owner_user_id
owners = defaultdict(list)
for prop in properties:
    owners[prop["owner_user_id"]].append(prop)


✅ Step 2: Generate landlord profiles from grouped data

In [8]:
from dateutil.parser import parse as parse_datetime

def generate_landlord_profile(owner_id, owned_props):
    # Derive preferred locations
    preferred_locations = list({p["location"] for p in owned_props if p["location"]})

    # Parse datetime strings robustly
    created_times = [p["created_at"] for p in owned_props if p["created_at"]]
    created_times_dt = [parse_datetime(t) for t in created_times]
    joined_at = min(created_times_dt) if created_times_dt else datetime.utcnow()

    return {
        "user_id": owner_id,
        "full_name": faker.name(),
        "phone_number": faker.phone_number(),
        "verified": False,
        "bio": faker.text(max_nb_chars=100),
        "profile_image_url": None,
        "joined_at": joined_at.isoformat(),
        "preferred_locations": preferred_locations
    }


landlord_profiles = [
    generate_landlord_profile(owner_id, props)
    for owner_id, props in owners.items()
]

print(f"Generated {len(landlord_profiles)} landlord profiles.")


Generated 642 landlord profiles.


✅ Step 3: Batch insert into landlord_profile table

In [9]:
from math import ceil

BATCH_SIZE = 50
num_batches = ceil(len(landlord_profiles) / BATCH_SIZE)

for i in range(num_batches):
    batch = landlord_profiles[i * BATCH_SIZE : (i + 1) * BATCH_SIZE]
    client.table("landlord_profile").insert(batch).execute()
    print(f"Inserted batch {i+1}/{num_batches} - {len(batch)} records.")


Inserted batch 1/13 - 50 records.
Inserted batch 2/13 - 50 records.
Inserted batch 3/13 - 50 records.
Inserted batch 4/13 - 50 records.
Inserted batch 5/13 - 50 records.
Inserted batch 6/13 - 50 records.
Inserted batch 7/13 - 50 records.
Inserted batch 8/13 - 50 records.
Inserted batch 9/13 - 50 records.
Inserted batch 10/13 - 50 records.
Inserted batch 11/13 - 50 records.
Inserted batch 12/13 - 50 records.
Inserted batch 13/13 - 42 records.


In [10]:
count_resp = client.table("landlord_profile").select("user_id", count="exact").execute()
print(f"Total landlord profiles in DB: {count_resp.count}")


Total landlord profiles in DB: 642
